In [ ]:
#import tensorflow as tf
import matplotlib. pyplot as plt
import keras
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import load_model
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

Using TensorFlow backend.


In [ ]:
df=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1")

In [ ]:
df

In [ ]:
decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
def decode_sentiment(label):
    return decode_map[int(label)]

df['0'] = df['0'].apply(lambda x: decode_sentiment(x))

In [4]:
data=df[["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D","0"]]

In [5]:
data['0'].factorize([0,4])

(array([0, 0, 0, ..., 1, 1, 1], dtype=int64),
 Int64Index([0, 4], dtype='int64'))

In [6]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
data.dtypes

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    object
0                                                                                                                       int64
dtype: object

In [7]:
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
import re
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

data["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"] = df["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"].apply(lambda x: preprocess(x))

C:\Users\aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [8]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"].values)
X = tokenizer.texts_to_sequences(data["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"].values)
X = pad_sequences(X)

In [9]:
Y = pd.get_dummies(data['0']).values
Y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

print(Y_train.shape,
X_train.shape)

(1071999, 2) (1071999, 48)


In [11]:
from keras.layers import SpatialDropout1D
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
batch_size = 32
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

model.fit(X_train, Y_train, epochs = 10, validation_data=(X_test, Y_test),batch_size=batch_size, verbose = 1,callbacks=callbacks)

C:\Users\aditya\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1071999 samples, validate on 528000 samples
Epoch 1/5
1071999/1071999 [==============================] - 3776s 4ms/step - loss: 0.4955 - accuracy: 0.7562 - val_loss: 0.4822 - val_accuracy: 0.7653
Epoch 2/5
1071999/1071999 [==============================] - 5339s 5ms/step - loss: 0.4816 - accuracy: 0.7656 - val_loss: 0.4765 - val_accuracy: 0.7680
Epoch 3/5
1071999/1071999 [==============================] - 3752s 4ms/step - loss: 0.4770 - accuracy: 0.7678 - val_loss: 0.4744 - val_accuracy: 0.7695
Epoch 4/5
1071999/1071999 [==============================] - 3043s 3ms/step - loss: 0.4744 - accuracy: 0.7691 - val_loss: 0.4730 - val_accuracy: 0.7708
Epoch 5/5
1071999/1071999 [==============================] - 3171s 3ms/step - loss: 0.4727 - accuracy: 0.7704 - val_loss: 0.4732 - val_accuracy: 0.7707


In [13]:
twt=["You are a very bad person fuck you"]
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=48, dtype='int32', value=0)

twt1=["You are a life saver man really appreciate your hard work"]
twt1 = tokenizer.texts_to_sequences(twt1)
twt1 = pad_sequences(twt1, maxlen=48, dtype='int32', value=0)

In [14]:
model.predict(twt)

array([[0.7572938 , 0.24270621]], dtype=float32)

In [15]:
print("You are a very bad person fuck you")
if np.argmax(model.predict(twt)[0])==1:
    print('Positive sentence')
else:
    print('Negative sentence')


You are a very bad person fuck you
Negative sentence


In [16]:
print("You are a life saver man really appreciate your hard work")
if np.argmax(model.predict(twt1)[0])==1:
    print('Positive sentence')
else:    
    print('Negative sentence')

You are a life saver man really appreciate your hard work
Positive sentence


In [17]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
model.save('sentiment.h5')    

In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)